In [1]:
!pip install python-pptx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.1 MB 8.0 MB/s 
     |████████████████████████████████| 149 kB 50.0 MB/s 
  Created wheel for python-pptx: filename=python_pptx-0.6.21-py3-none-any.whl size=470951 sha256=a2241ed9698816927a558ad1248aab144dbddd37a536f8f057674f7a854504bd
  Stored in directory: /root/.cache/pip/wheels/a7/ab/f4/52560d0d4bd4055e9261c6df6e51c7b56c2b23cca3dee811a3
Successfully built python-pptx


In [2]:
#General imports
import keras
import tensorflow as tf
import matplotlib.pyplot as plt

from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint

from pathlib import Path
from datetime import datetime
import json
import os

In [3]:
cd /content/drive/MyDrive/Giraffa Analytics/MRI-Blur-Detection

/content/drive/MyDrive/Giraffa Analytics/MRI-Blur-Detection


In [4]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%autoreload 2

from Model.DataGenerator import DataGenerator
from Model.CNN_2D_model import get_uncompiled_model
from Model.project_scripts import train_validation_test_split
from Model.CustomCallbacks import PlottingCallback
from Model.CustomCallbacks import PptxReportCallback

In [5]:
train_validation_test_split()

({'test': ['Data/preprocessed slices/slice_150_NC239_motion.npy',
   'Data/preprocessed slices/slice_150_NC239_nomotion.npy',
   'Data/preprocessed slices/slice_150_NC243_motion.npy',
   'Data/preprocessed slices/slice_150_NC243_nomotion.npy',
   'Data/preprocessed slices/slice_151_NC239_motion.npy',
   'Data/preprocessed slices/slice_151_NC239_nomotion.npy',
   'Data/preprocessed slices/slice_151_NC243_motion.npy',
   'Data/preprocessed slices/slice_151_NC243_nomotion.npy',
   'Data/preprocessed slices/slice_152_NC239_motion.npy',
   'Data/preprocessed slices/slice_152_NC239_nomotion.npy',
   'Data/preprocessed slices/slice_152_NC243_motion.npy',
   'Data/preprocessed slices/slice_152_NC243_nomotion.npy',
   'Data/preprocessed slices/slice_153_NC239_motion.npy',
   'Data/preprocessed slices/slice_153_NC239_nomotion.npy',
   'Data/preprocessed slices/slice_153_NC243_motion.npy',
   'Data/preprocessed slices/slice_153_NC243_nomotion.npy',
   'Data/preprocessed slices/slice_154_NC239_mot

In [6]:
training_generator = DataGenerator('train')
validation_generator = DataGenerator('validation')
testing_generator = DataGenerator('test')

In [7]:
gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

In [8]:
# Design model
#GET MODEL NAME (the name is given by the time it started fitting)
CURRENT_TIME = datetime.now().strftime("%d-%m-%Y %H:%M")

#Create folder for the callbacks
os.mkdir(f'Callbacks/{CURRENT_TIME}')

model = get_uncompiled_model()

In [9]:
#We compile the model
optimizer = tf.keras.optimizers.Adagrad()
model.compile(
    #change these
    optimizer= optimizer,  # Optimizer
    # Loss function to minimize
    loss=keras.losses.BinaryCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.Accuracy(),keras.metrics.BinaryAccuracy(),keras.metrics.BinaryCrossentropy()],
)

optimizer_json = optimizer.get_config()

with open(f'Callbacks/{CURRENT_TIME}/optimizer.txt','w') as optimizer_file:
    json.dump(optimizer_json, optimizer_file)

In [10]:
#We save the summary of the model as a .txt file (PptxReportCallback will make use of it)
with open(f'Callbacks/{CURRENT_TIME}/model_summary.txt','w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    model.summary(print_fn=lambda x: fh.write(x + '\n'), line_length=120)

In [11]:
#Save config
config = model.get_config() #it is a dictionary which can be saved as a .json
with open(f'Callbacks/{CURRENT_TIME}/config.json', 'w') as config_file:
    json.dump(config, config_file)

#CALLBACKS
#CSV LOGGER
logger_path = f'Callbacks/{CURRENT_TIME}/training.log'
logger_dir = os.path.dirname(logger_path)
csv_logger = CSVLogger(logger_path, append = False)

#ModelCheckpoint
checkpoint_path = f'Callbacks/{CURRENT_TIME}/model_checkpoint/training/'+'cp-{epoch:04d}.ckpt'
os.mkdir(f'Callbacks/{CURRENT_TIME}/model_checkpoint')
os.mkdir(f'Callbacks/{CURRENT_TIME}/model_checkpoint/training')
model_checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True)

#PlottingCallback --custom callback
loss_acc_plotter = PlottingCallback(f'Callbacks/{CURRENT_TIME}')

#PpptxCallback
pptx_writer = PptxReportCallback(folder_path='Callbacks', model_dir=f'Callbacks/{CURRENT_TIME}')

#The callbacks will be executed in the order given below
model_callbacks = [csv_logger, model_checkpoint, loss_acc_plotter, pptx_writer]

In [13]:
#We fit the model
model.fit(training_generator,
          validation_data=validation_generator,
          verbose=2,
          epochs=150,
          callbacks = model_callbacks
          )

Epoch 1/150

Epoch 1: val_accuracy did not improve from 0.50670
81/81 - 9s - loss: 0.0024 - accuracy: 0.5019 - binary_accuracy: 0.5019 - binary_crossentropy: 0.0024 - val_loss: 2.3095 - val_accuracy: 0.5000 - val_binary_accuracy: 0.5000 - val_binary_crossentropy: 2.3095 - 9s/epoch - 115ms/step
Epoch 2/150


KeyboardInterrupt: ignored